In [3]:
import re
import markovify
from sentence_transformers import SentenceTransformer, util
import json
import pyphen

In [4]:
text = open('lyrics/vindicator.txt', 'rb').read().decode(encoding='utf-8')
text = re.sub('\[.*\]', '', text)
print(text[:500])

chickens in the sink
crack on the stove
triple beam for the weight
cut the middle man tryna get 8 white bitches
snorting off the plate coke from colombia
riding round in the benz looking like a trap star
flipping raw day and night
tryna get my money right naw i ain't living rite
fucking bitches all night kick 'em out the next day
everyday is pay day
hit me with a half bring it back in two days
living in califronia by the ocean
condo got em open
long dick stroke em
i send 'em to the


In [5]:
bars =  text.split('\n')
bars = [[re.sub(r'[^a-zA-Z0-9]', '', word) for word in bar.split(' ')] for bar in bars]

word_text = re.sub('\n', ' ', text)
words = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in word_text.split(' ')]
words = [word for word in words if word != '']

max = 0
for bar in bars:
    if len(bar) > max: max = len(bar)
print('Longest bar: ', max)

bar_lengths = [len(bar) for bar in bars]
print('average bar length: ', round(sum(bar_lengths)/len(bar_lengths), 1), ' words')

cutoff = 15
print(f'prop of bars less then {cutoff} words long: ', round(len([bar for bar in bar_lengths if bar < cutoff]) / len(bar_lengths), 2))

Longest bar:  37
average bar length:  8.3  words
prop of bars less then 15 words long:  0.98


In [6]:
#unique, frequency = np.unique(np.array(words), return_counts=True)

freq_dict = {}

for index, word in enumerate(words):
    if word in freq_dict:
        freq_dict[word] = freq_dict[word] + 1
    else: freq_dict[word] = 1

vocab = sorted(list(freq_dict.keys()))

top = sorted(freq_dict.items(), key=lambda x:x[1], reverse=True)[:40]


In [7]:
markov_text = markovify.NewlineText(text, state_size=3)

In [8]:
for i in range(10):
    next_bar = None
    while next_bar is None:
        next_bar = markov_text.make_sentence(tries = 20)
    print(next_bar)

but secrets of the spirit world and how to be great so those classes can wait
happened on sunday we should have known i was sick when i wasn't white enough
no i am only talking about the same shit again
if you could do it or that i believe in
it's a place i got to fight one
fuck what you might think i'm too aggressive
heathen it's a struggle just to be fake
coughing up blood like what the fuck you mean i been up on an ultralight beam
yet the cuter version yet i knew you since we was in we was in high school none of these niggas
and you can blame it on the air


In [9]:
model = SentenceTransformer('stsb-roberta-large')

In [10]:
stop_words = json.load(open('utils/stop_words_english.json'))
stop_words.append('__BEGIN__')
stop_words.append('__END__')

In [11]:
def similarity(first, second):
    first_vec = model.encode(first, convert_to_tensor=True)
    second_vec = model.encode(second, convert_to_tensor=True)
    return util.pytorch_cos_sim(first_vec, second_vec).item()

In [12]:
# total_keywords = len(markov_text.chain.model)
# ctr = 0
# for keywords, next_words in markov_text.chain.model.items():

#     print(f'{ctr}/{total_keywords}', end = '\r')
#     ctr = ctr + 1
    
#     semantic_words = []

#     for keyword in keywords:
#         if not re.sub(r'[^a-zA-Z0-9]', '', keyword).lower() in stop_words:
#             semantic_words.append(keyword)

#     if len(semantic_words) > 0:
#         key_string = ' '.join(semantic_words)

#         for word, freq in next_words.items():
#             if not re.sub(r'[^a-zA-Z0-9]', '', word).lower() in stop_words:
#                 next_words[word] = round(next_words[word] * similarity(key_string, word) * 1000)
#             else: next_words[word] = next_words[word] * 750

In [13]:
# model_json = markov_text.to_json()
# with open('markov_model2.json', 'w') as f:
#     json.dump(model_json, f)

In [14]:
saved_model = markovify.Text.from_json(json.load(open(('markov_model.json'))))

In [15]:
saved_model.chain.model

{('___BEGIN__', '___BEGIN__', '___BEGIN__'): {'Look,': 6,
  'But': 635,
  "Something's": 2,
  'Just': 178,
  'Like': 93,
  'If': 192,
  'Big': 9,
  "I'm": 601,
  'You': 608,
  'All': 97,
  'Now': 207,
  'They': 94,
  'I': 1494,
  'My': 220,
  'Got': 43,
  'Made': 6,
  'Ever': 17,
  'With': 101,
  'This': 165,
  'Packing': 4,
  'Backpack': 1,
  'And': 1025,
  'Over': 5,
  'Only': 11,
  'Feel': 10,
  'Rappers': 5,
  "Here's": 14,
  "It's": 259,
  'For': 64,
  'Let': 45,
  'Everybody': 20,
  'Immortality': 1,
  'Well,': 30,
  'Simply': 1,
  'Hit': 22,
  'MCs': 1,
  'Me?': 2,
  'Lakim': 1,
  'Yella,': 1,
  'Inspired': 1,
  'Blow': 7,
  'To': 153,
  'Roll': 3,
  'On': 27,
  'Off': 5,
  'Tell': 11,
  'Little': 17,
  "You're": 112,
  'Oy': 1,
  "That's": 111,
  'Hey,': 4,
  "I'mma": 28,
  'Never': 36,
  'Get': 55,
  'Basically': 1,
  'The': 261,
  'Dale': 1,
  'Kneel': 1,
  'So': 406,
  'We': 150,
  'In': 111,
  'What': 92,
  'At': 18,
  'Somewhere': 3,
  'Enough': 4,
  'Maybe': 81,
  'Was': 

In [35]:
next_semantic_bar = ''
bars = []

for i in range(8):

    prev_semantic_bar = next_semantic_bar
    next_bar = None
    corr = 0

    while next_bar is None or corr < .3:

        next_bar = saved_model.make_sentence(tries=20)
        if not next_bar is None: 
            next_semantic_bar = ' '.join([word for word in next_bar.split(' ') if re.sub(r'[^a-zA-Z0-9]', '', word).lower() not in stop_words]) 
            corr = similarity(prev_semantic_bar, next_semantic_bar)
        
    bars.append(next_bar)
    print(next_bar)

But there's just something about our love
I've been to the top
I know sometimes things may not always make sense to you right now your not gonna like it
I can see the truth
I think that would fit with the definition of what a concrete chin is
Nothing nobody told you when you stepping on turf, Q
Don't front, you know I ripped all of your pictures off the wall with your two left feet
Oh, bitch, I don't know how else to put it out once we light it


In [36]:
pyphen.language_fallback('en_US_someVariant')
dic = pyphen.Pyphen(lang='en_US')
dic.inserted('hello')

phonetic_bars = []
for bar in bars:
    phonetic_bars.append(' '.join([dic.inserted(word) for word in bar.split(' ')]))

print(phonetic_bars)

["But there's just some-thing about our love", "I've been to the top", 'I know some-times things may not al-ways make sense to you right now your not gonna like it', 'I can see the truth', 'I think that would fit with the def-i-ni-tion of what a con-crete chin is', 'Noth-ing no-body told you when you step-ping on tur-f, Q', "Don't fron-t, you know I ripped all of your pic-tures off the wall with your two left feet", "Oh, bitch, I don't know how else to put it out once we light it"]
